In [10]:
from sklearn.datasets import load_breast_cancer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt

import pandas as pd

import numpy as np

In [9]:
# Set random state for project
RD_STATE = 1

# Load breast cancer dataset
bc_data = load_breast_cancer()

# Pisahkan menjadi training set (80%) dan testing set (20%)
bc_X_train, bc_X_test, bc_y_train, bc_y_test = train_test_split(bc_data.data, bc_data.target, test_size=0.2, random_state=RD_STATE)

In [3]:
# Load play tennis dataset
pt_df_raw = pd.read_csv("play_tennis.csv")
pt_df = pt_df_raw.copy()

# Label encoding
# Karena data play tennis bentuknya kategorikal nominal,
# maka harus diubah menjadi bentuk numerik yang bisa diproses oleh model machine learning numerik
le = LabelEncoder()
pt_classes = {}
for column in pt_df.columns:
    le.fit(pt_df[column])
    pt_df[column] = le.transform(pt_df[column])
    pt_classes[column] = le.classes_
pt_X_train, pt_X_test, pt_y_train, pt_y_test = train_test_split(pt_df[['outlook', 'temp', 'humidity', 'wind']], pt_df[['play']], test_size=0.2, random_state=RD_STATE)

In [4]:
# Logistic Regression

# Data preprocessing: normalisasi data agar setiap feature memiliki orde magnituda yang sama
bc_scaler = StandardScaler().fit(bc_data.data)
bc_X_train_scaled = bc_scaler.transform(bc_X_train)
bc_X_test_scaled = bc_scaler.transform(bc_X_test)

pt_scaler = StandardScaler().fit(pt_df[['outlook', 'temp', 'humidity', 'wind']])
pt_X_train_scaled = pt_scaler.transform(pt_X_train)
pt_X_test_scaled = pt_scaler.transform(pt_X_test)

# Fit classifier
bc_lr_clf = LogisticRegression(random_state=RD_STATE).fit(bc_X_train_scaled, bc_y_train)

pt_lr_clf = LogisticRegression(random_state=RD_STATE).fit(pt_X_train_scaled, pt_y_train['play'])

# Print accuracy dan nilai f1
print("Logistic Regression Performance:")
print("• Breast cancer dataset")
print("\tAccuracy: ", bc_lr_clf.score(bc_X_test_scaled, bc_y_test))
print("\tf1_score: ", f1_score(bc_y_test, bc_lr_clf.predict(bc_X_test_scaled)))
print("• Play tennis dataset")
print("\tAccuracy: ", pt_lr_clf.score(pt_X_test_scaled, pt_y_test['play']))
print("\tf1_score: ", f1_score(pt_y_test['play'], pt_lr_clf.predict(pt_X_test_scaled)))

Logistic Regression Performance:
• Breast cancer dataset
	Accuracy:  0.9736842105263158
	f1_score:  0.9793103448275863
• Play tennis dataset
	Accuracy:  0.6666666666666666
	f1_score:  0.8


Logistic regression untuk dua kelas melibatkan fungsi logit yang menggunakan nilai dot product dari vektor bobot b dengan vektor inputs x ditambah bias sebagai parameter exponennya. Dalam bentuk rumus:

 • y_pred = 1/(1+exp(-(∑(b_i.x_i)+b_0)))

Training loop pada model logistic regression merupakan proses perhitungan cost dari hasil prediksi terhadap label sebetulnya, kemudian memperbarui nilai vektor bobot b, misal dengan stochastic gradient descent menggunakan rumus b_j = b_j + a * (y_i-p_i) * x_ij, sehingga prediksi training set converge terhadap nilai asli.

Kinerja logistic regression pada dataset breast cancer cukup baik, accuracy 97% dan nilai f1 98%. Sementara itu, play tennis dataset memiliki accuracy 66.7% yang cukup buruk dan nilai f1 80% yang rata-rata. Namun ini karena ukuran test set play tennis cukup kecil, hanya 3, dan training set yang relatif kecil, hanya 11. Sebagai perbandingan, dataset breast cancer memiliki ukuran training set 455 dan test set 114. Oleh karena itu, terdapat noise yang mengganggu hasil analisis kinerja. Training set kecil menyebabkan kemungkinan overfit meningkat, dan testing set kecil berarti variance dataset condong lebih besar. Jika dataset lebih besar, maka accuracy classifier pada dataset play tennis kemungkinan berubah menjadi tidak lagi 66.7%.

In [55]:
# Neural Network: Multi-Layer Perceptron Model

# Fit classifier
bc_mlp_clf = MLPClassifier(random_state=RD_STATE, activation='relu', max_iter=1000, hidden_layer_sizes=(100,)).fit(bc_X_train, bc_y_train)
1
pt_mlp_clf = MLPClassifier(random_state=RD_STATE, activation='relu', max_iter=1000, hidden_layer_sizes=(100,)).fit(pt_X_train, pt_y_train['play'])

# Print accuracy dan nilai f1
print("Neural Network Performance:")
print("• Breast cancer dataset")
print("\tAccuracy: ", bc_mlp_clf.score(bc_X_test, bc_y_test))
print("\tf1_score: ", f1_score(bc_y_test, bc_mlp_clf.predict(bc_X_test)))
print("• Play tennis dataset")
print("\tAccuracy: ", pt_mlp_clf.score(pt_X_test, pt_y_test['play']))
print("\tf1_score: ", f1_score(pt_y_test['play'], pt_mlp_clf.predict(pt_X_test)))

Neural Network Performance:
• Breast cancer dataset
	Accuracy:  0.9473684210526315
	f1_score:  0.9594594594594595
• Play tennis dataset
	Accuracy:  1.0
	f1_score:  1.0


In [56]:
bc_mlp_params = bc_mlp_clf.get_params()
pt_mlp_params = pt_mlp_clf.get_params()

print("Parameter MLP dataset breast cancer:")
print("• Activation function\t:", bc_mlp_params['activation'])
print("• Learning rate\t\t:", bc_mlp_params['learning_rate_init'])
print("• Max iterasi\t\t:", bc_mlp_params['max_iter'])
print("• Jumlah hidden layer\t:", bc_mlp_params['hidden_layer_sizes'][0])
print("\nParameter MLP dataset play tennis:")
print("• Activation function\t:", pt_mlp_params['activation'])
print("• Learning rate\t\t:", pt_mlp_params['learning_rate_init'])
print("• Max iterasi\t\t:", pt_mlp_params['max_iter'])
print("• Jumlah hidden layer\t:", pt_mlp_params['hidden_layer_sizes'][0])

Parameter MLP dataset breast cancer:
• Activation function	: relu
• Learning rate		: 0.001
• Max iterasi		: 1000
• Jumlah hidden layer	: 100

Parameter MLP dataset play tennis:
• Activation function	: relu
• Learning rate		: 0.001
• Max iterasi		: 1000
• Jumlah hidden layer	: 100


Multilayer Perceptron merupakan jenis Artificial Neural Network, sebuah model machine learning yang menggunakan node perceptron sebagai blok pembangun modelnya. Dalam MLP, terdapat tiga jenis layer node: input layer, hidden layer, dan output layer. Input layer berjumlah sebanyak parameter input, kemudian nilai setiap node pada layer berikutnya dipengaruhi oleh node-node input berdasarkan bobot tertentu. Untuk suatu node yang dipengaruhi oleh nilai node pada layer sebelumnya x_ij, nilai terbobotnya merupakan ∑(∑(w_ij * x_ij) + b_i). Nilai ini kemudian dilalui sebuah fungsi aktivasi, yang digunakan untuk memetakan nilai terbobot ke suatu rentang tertentu. Hasil prediksi dari output layer digunakan untuk menghitung cost dan memperbarui nilai bobot layer terakhir sebelum output layer. Nilai baru pada layer tersebut digunakan kembali untuk menghitung cost dan memberbarui nilai bobot layer sebelumnya, dan seterusnya hingga sampai ke layer pertama.

Terdapat beberapa jenis fungsi aktivasi seperti fungsi sigmoid (seperti yang digunakan di logistic regression), fungsi hyperbolic tan, ReLU, dan fungsi identitas. Fungsi sigmoid memiliki bentuk S horizontal yang memetakan nilai terbobot menjadi 0 atau 1. Fungsi tanh memetakan nilai terbobot menjadi -1 atau 1, kemudian fungsi ReLU memetakan nilai negatif menjadi 0 dan nilai positif tidak diubah, sementara fungsi identitas tidak mengubah nilai terbobot.

Dalam tugas ini, fungsi ReLU digunakan. Fungsi ReLU cukup umum untuk digunakan dalam MLP karena menghindari masalah numerik yang dialami fungsi sigmoid dan tanh saat melakukan gradient descent. Berikut rumus fungsi ReLU:

• x_ij = max(0, x_ij)

Kinerja MLP pada dataset breast cancer dan play tennis cukup baik, dengan accuracy 94.7% dan nilai f1 95.9% pada dataset breast cancer serta accuracy 100% dan nilai f1 100% pada dataset play tennis. Namun karena dataset play tennis cukup kecil, maka terdapat kemungkinan bahwa terdapat noise yang mempengaruhi kinerja classifier MLP-nya.

In [6]:
# Support Vector Machine: C-Support Vector Classification

# Data preprocessing: normalisasi data agar setiap feature memiliki orde magnituda yang sama
bc_scaler = StandardScaler().fit(bc_data.data)
bc_X_train_scaled = bc_scaler.transform(bc_X_train)
bc_X_test_scaled = bc_scaler.transform(bc_X_test)

pt_scaler = StandardScaler().fit(pt_df[['outlook', 'temp', 'humidity', 'wind']])
pt_X_train_scaled = pt_scaler.transform(pt_X_train)
pt_X_test_scaled = pt_scaler.transform(pt_X_test)

# Fit classifier
bc_svc_clf = SVC(random_state=RD_STATE, gamma='auto').fit(bc_X_train_scaled, bc_y_train)

pt_svc_clf = SVC(random_state=RD_STATE, gamma='auto').fit(pt_X_train_scaled, pt_y_train['play'])

# Print accuracy dan nilai f1
print("SVM Performance:")
print("• Breast cancer dataset")
print("\tAccuracy: ", bc_svc_clf.score(bc_X_test_scaled, bc_y_test))
print("\tf1_score: ", f1_score(bc_y_test, bc_svc_clf.predict(bc_X_test_scaled)))
print("• Play tennis dataset")
print("\tAccuracy: ", pt_svc_clf.score(pt_X_test_scaled, pt_y_test['play']))
print("\tf1_score: ", f1_score(pt_y_test['play'], pt_svc_clf.predict(pt_X_test_scaled)))

SVM Performance:
• Breast cancer dataset
	Accuracy:  0.9736842105263158
	f1_score:  0.9793103448275863
• Play tennis dataset
	Accuracy:  1.0
	f1_score:  1.0


In [59]:
bc_svc_params = bc_svc_clf.get_params()
pt_svc_params = pt_svc_clf.get_params()

print("Parameter MLP dataset breast cancer:")
print("• Penalty parameter\t:", bc_svc_params['C'])
print("• Kernel\t\t:", bc_svc_params['kernel'])
print("\nParameter MLP dataset play tennis:")
print("• Penalty parameter\t:", pt_svc_params['C'])
print("• Kernel\t\t:", pt_svc_params['kernel'])

Parameter MLP dataset breast cancer:
• Penalty parameter	: 1.0
• Kernel		: rbf

Parameter MLP dataset play tennis:
• Penalty parameter	: 1.0
• Kernel		: rbf
